https://www.tensorflow.org/get_started/get_started

## Tensors

### In the tutorial
```The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's rank is its number of dimensions.```
### From wikipedia
```In mathematics, tensors are geometric objects that describe linear relations between geometric vectors, scalars, and other tensors. Elementary examples of such relations include the dot product, the cross product, and linear maps. Geometric vectors, often used in physics and engineering applications, and scalars themselves are also tensors.```

## Computational Graphs

```A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output.```
```One type of node is a constant. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally.```
Its value can never change, and it gets initialized as soon as you create it (we get more on that later).

In [1]:
from __future__ import print_function
import tensorflow as tf

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


```A session encapsulates the control and state of the TensorFlow runtime.```
You can run a computational graph within a session, at which point everything
within the computational graph gets evaluated:

In [2]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


Another type of node is an operation node. You can get fancier by combining
constant nodes with operation nodes.

In [3]:
node3 = tf.add(node1, node2)
print("node3: ", node3) # this prints the object and shape gobbledygook
print("sess.run(node3):", sess.run(node3)) # this actually evaluates

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


Next they explain that TensorBoards are basically just visualizations of the Flow / ComputationalGraph / DAG. They look much like Airflow or KNIME.
Weirdly, although they're called "Flow", the TensorBoard examples here don't have arrows, and go bottom to top, which is visually very counter-intuitive to me. But, maybe they're meant to be flexible enough to go both directions
(like neural networks go forwards and then backprop backwards) ...so maybe the point is to not have one "obvious" direction and one "counter-intuitive" direction.

If you wanted to provide values for nodes 1 and 2 that change between runs and you don't want to hardcode that, you can make a placeholder node in your graph. When you run the session, you provide the values for those nodes.
like arguments. (See further down).
You must provide the datatype when you make a placeholder, here's the list of datatypes: https://www.tensorflow.org/versions/r0.12/resources/dims_types

In [7]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # alternative to tf.add(a, b)
print(a, b)

Tensor("Placeholder_6:0", dtype=float32) Tensor("Placeholder_7:0", dtype=float32)


Then the tutorial says to use the feed_dict argument to get the values you into the placeholder nodes. The tutorial uses a fixed argument here, but the docs have a keyword argument. Turns out they both work.

In [10]:
print(sess.run(adder_node, {a: 3, b: 4.5})) # works, returns a floats
print(sess.run(adder_node, feed_dict={a: 3, b: 4.5})) # also works, also returns float
print(sess.run(adder_node, {a: [1, 3, 4], b: [2, 4, 4]})) # returns numpy array

7.5
7.5
[ 3.  7.  8.]


Above, we declared a and b to be floats. What's going on in that last example is that we can provide multiple values of them at once. With one call to `sess.run` you can get results from multiple sets of inputs. More examples:

In [12]:
print(sess.run(adder_node, {a: 8, b: 9})) # returns float
print(sess.run(adder_node, {a: 1/2, b: 6})) # returns float
print(sess.run(adder_node, {a: [1, 3, 4], b: [2, 4, [4]]})) # no good!

17.0
6.0


ValueError: setting an array element with a sequence.

That last guy isn't a boxy enough array of arrays, so what would
adding those mean?
Next we're going to get fancier with more nodes.

In [13]:
add_and_triple = adder_node * 3.
print(add_and_triple)
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

Tensor("mul:0", dtype=float32)
22.5


Here's a good summary from the tutorial:
```In machine learning we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters to a graph.```
Key point: the constant nodes or placeholder nodes won't change value throughout the run of a session. If we want something that will change value, we'll use variable nodes.

In [16]:
W = tf.Variable([.3], dtype=tf.float32)
print(W)
b = tf.Variable([-.3], dtype=tf.float32)
# my_variable = tf.get_variable("my_variable", [1, 2, 3])
x = tf.placeholder(tf.float32)
linear_model = W*x + b

<tf.Variable 'Variable_2:0' shape=(1,) dtype=float32_ref>


In TensorFlow, variables aren't initialized until you say that they should be. Why? They don't say in the tutorial, but here's from the docs:
```Explicit initialization is otherwise useful because it allows you not to
rerun potentially expensive initializers when reloading a model from a
checkpoint as well as allowing determinism when randomly-initialized
variables are shared in a distributed setting.```
(https://www.tensorflow.org/programmers_guide/variables)

Here's how you say that the variables should all be initialized:

In [20]:
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


So far we have:
* created a trainable model
* run the model

Next we'll want to evaluate the model against some training data. For that, we'll need to:
* create something to hold the correct labels / classifications / output values
* write an error function. in this case ```we'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data.```

In [21]:
y = tf.placeholder(tf.float32) # this will hold the correct output values
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1,2,3,4], y: [0, -1, -2, -3]}))

23.66


In [22]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


## tf.train API

```the whole point of machine learning is to find the correct model parameters automatically.```

To that end, TensorFlow provides **optimizers** that slowly change each variable to minimize loss and approach the correct answer. 

According to the tutorial, "the simplest optimizer is **gradient descent**. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable." 

That is not fun to do by hand, so aren't you glad that's taken care of for you?

In [24]:
optimizer = tf.train.GradientDescentOptimizer(0.01) # that's the magnitude of the learning step
train = optimizer.minimize(loss)

In [29]:
sess.run(init) # we fixed the values above so now we're unfixing
for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print(sess.run([W, b]))

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]
W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


![](https://www.tensorflow.org/images/getting_started_final.png)

## tf.estimator

This is a TensorFlow library that includes features to manage data sets, training, evaluating, etc. so that more of this is abstracted away as something we need to worry about. An **estimator** is basically a front-end/shortcut to train/fit the model and evaluate it (using statistical inference) using a particular model.

In [30]:
import numpy as np
import tensorflow as tf

**Step 1: declare a list of features** There are many kinds of features, and you could have a very long list if you chose.

In [31]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

**Step 2: choose an estimator**. There are lots of ready-defined types to use, in the tutorial they use a linear regression.

In [32]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114a98f50>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/2j/604kc25x1fqbc542297fpq1c0000gn/T/tmpTsHUX4', '_save_summary_steps': 100}


**Step 3: set up our training and test data sets**. tf.estimator provides helper functions for this. Note that `num_epochs` is how many batches of data you want.

In [35]:
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

**Step 4: train the model**. Here we're specifying 1000 training steps or iterations.

In [36]:
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/2j/604kc25x1fqbc542297fpq1c0000gn/T/tmpTsHUX4/model.ckpt.
INFO:tensorflow:loss = 15.0, step = 1
INFO:tensorflow:global_step/sec: 629.565
INFO:tensorflow:loss = 0.578425, step = 101 (0.173 sec)
INFO:tensorflow:global_step/sec: 679.764
INFO:tensorflow:loss = 0.0305686, step = 201 (0.135 sec)
INFO:tensorflow:global_step/sec: 818.343
INFO:tensorflow:loss = 0.00842404, step = 301 (0.122 sec)
INFO:tensorflow:global_step/sec: 936.461
INFO:tensorflow:loss = 0.00361122, step = 401 (0.107 sec)
INFO:tensorflow:global_step/sec: 977.47
INFO:tensorflow:loss = 0.000888144, step = 501 (0.102 sec)
INFO:tensorflow:global_step/sec: 1123.65
INFO:tensorflow:loss = 0.000176936, step = 601 (0.089 sec)
INFO:tensorflow:global_step/sec: 1073.38
INFO:tensorflow:loss = 3.17232e-05, step = 701 (0.093 sec)
INFO:tensorflow:global_step/sec: 1099.35
INFO:tensorflow:loss = 4.2329e-06, step = 801 (0.091 sec)
INFO:tenso

**Step 5: evaluate performance.**

In [37]:
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-05-01:55:05
INFO:tensorflow:Restoring parameters from /var/folders/2j/604kc25x1fqbc542297fpq1c0000gn/T/tmpTsHUX4/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-05-01:55:07
INFO:tensorflow:Saving dict for global step 1000: average_loss = 5.19178e-08, global_step = 1000, loss = 2.07671e-07
INFO:tensorflow:Starting evaluation at 2017-12-05-01:55:07
INFO:tensorflow:Restoring parameters from /var/folders/2j/604kc25x1fqbc542297fpq1c0000gn/T/tmpTsHUX4/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-05-01:55:08
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00254646, global_step = 1000, loss = 0.0101858
train metrics: {'average_loss': 5.1917795e-08, 'global_step': 1000, 'loss': 2.0767118e-07}
eval metrics: {'average_loss': 0.0025464611, 'global_step': 1000, 'loss': 0.010185844}


### Custom models

The tutorial gives a walkthrough of how to create a custom model, if the model (like `LinearRegressor`) isn't built into the TF API (or if, say, you wanted to adapt the implementation of it for whatever reason.

The generic class is the `Estimator` class, so `LinearRegressor` is actual a sub-class of `Estimator`. But to make a custom model, you don't need to subclass `Estimator` -- you can just provide a function, `model_fn`, that tells the estimator `how it can evaluate predictions, training steps, and loss.`.

In [40]:
import numpy as np
import tensorflow as tf

**Step 1: declare a list of features** (same as above)

In [41]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

#### Step 2: define an estimator (here's where our code will be different from the above standard walkthrough)

In [44]:
def model_fn(features, labels, mode):
    # build a linear model
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    
    # define the loss ("loss sub-graph")
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # define training ("training sub-graph")
    global_step = tf.train.get_global_step()  # this tracks what training iteration we're on
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1)) # this just groups the two operations, loss minimizing and incrementing the step
    
    # EstimatorSpec is what connects the sub-graphs we just defined above
    # to get the ultimate functionality
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=y,
        loss=loss,
        train_op=train
    )

estimator = tf.estimator.Estimator(model_fn=model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11570f9d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/2j/604kc25x1fqbc542297fpq1c0000gn/T/tmp8yQJng', '_save_summary_steps': 100}


**Step 3: set up our training and test data sets** (same as above)

In [46]:
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

**Step 4: train the model** (same as above)

In [48]:
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/2j/604kc25x1fqbc542297fpq1c0000gn/T/tmp8yQJng/model.ckpt.
INFO:tensorflow:loss = 12.4964003622, step = 1
INFO:tensorflow:global_step/sec: 566.717
INFO:tensorflow:loss = 0.440538517976, step = 101 (0.129 sec)
INFO:tensorflow:global_step/sec: 853.045
INFO:tensorflow:loss = 0.0263902392921, step = 201 (0.117 sec)
INFO:tensorflow:global_step/sec: 984.28
INFO:tensorflow:loss = 0.0024488436504, step = 301 (0.102 sec)
INFO:tensorflow:global_step/sec: 958.323
INFO:tensorflow:loss = 8.65175725917e-05, step = 401 (0.104 sec)
INFO:tensorflow:global_step/sec: 916.876
INFO:tensorflow:loss = 4.00314831506e-06, step = 501 (0.110 sec)
INFO:tensorflow:global_step/sec: 949.045
INFO:tensorflow:loss = 1.74375072614e-06, step = 601 (0.105 sec)
INFO:tensorflow:global_step/sec: 961.899
INFO:tensorflow:loss = 8.49645655008e-08, step = 701 (0.104 sec)
INFO:tensorflow:global_step/sec: 955.566
INFO:tensorflow:l

**Step 5: evaluate performance** (same as above)

In [49]:
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-05-02:16:10
INFO:tensorflow:Restoring parameters from /var/folders/2j/604kc25x1fqbc542297fpq1c0000gn/T/tmp8yQJng/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-05-02:16:11
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.21545e-10
INFO:tensorflow:Starting evaluation at 2017-12-05-02:16:11
INFO:tensorflow:Restoring parameters from /var/folders/2j/604kc25x1fqbc542297fpq1c0000gn/T/tmp8yQJng/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-05-02:16:12
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101012
train metrics: {'loss': 1.2154536e-10, 'global_step': 1000}
eval metrics: {'loss': 0.010101224, 'global_step': 1000}
